# 5 Slepian Symmetry

In [6]:
using SpecialFunctions  # for besselj
using QuadGK            # for numerical integration quadgk
using Printf
using LinearAlgebra     # just for norm, if needed

## Bessel Function

Bessel Function of the First Kind, J_m(k), for integer m 

J_m(k) = sum_{n=0}^{∞} [ (-1)^n / (n! (m+n)!) ] * (k/2)^(m+2n) 

Also satisfies the symmetry condition J_{-m}(k) = (-1)^m J_m(k).\\

In [9]:
"""
    identity_sum_J(k, M)

Compute the partial sum S_M = J_0^2(k) + 2∑_{m=1 to M} J_m^2(k)
and compare it to 1. As M -> ∞, we expect S_M -> 1.
"""
function identity_sum_J(k::Real, M::Integer)
    term0 = besselj(0, k)^2
    # sum over m = 1..M
    term_sum = sum( besselj(m, k)^2 for m in 1:M )
    return term0 + 2*term_sum
end

"""
    bessel_expansion_2D(k, r, rprime, theta, thetaprime, M)

Compute the partial sum of 
  J_0(k||x - x'||)  ≈  J_0(kr) J_0(kr') + 2∑_{m=1..M} J_m(kr) J_m(kr') cos[m(θ - θ')].
Returns a tuple (lhs, rhs_M):

- lhs = J_0(k * distance(x, x'))
- rhs_M = partial sum up to m = M.
  
As M grows, rhs_M should converge to lhs.
"""
function bessel_expansion_2D(k::Real,
                             r::Real, rprime::Real,
                             theta::Real, thetaprime::Real,
                             M::Integer)
    # LHS: exact Bessel function on the distance
    # distance in Cartesian: x   = (r cosθ,   r sinθ)
    #                       x'  = (r'cosθ', r' sinθ')
    # norm(x - x') = sqrt(r^2 + r'^2 - 2 r r' cos(θ - θ'))
    dist = sqrt(r^2 + rprime^2 - 2r*rprime*cos(theta - thetaprime))
    lhs = besselj(0, k * dist)
    
    # RHS: partial sum
    rhs0 = besselj(0, k*r) * besselj(0, k*rprime)
    rhs_sum = 0.0
    for m in 1:M
        rhs_sum += besselj(m, k*r)*besselj(m, k*rprime)*cos(m*(theta - thetaprime))
    end
    rhs_M = rhs0 + 2*rhs_sum
    
    return lhs, rhs_M
end

bessel_expansion_2D

### Sanity check for Bessel Function

In [10]:
"""
    check_symmetry(m, k)

Check numerically that J_{-m}(k) = (-1)^m * J_m(k).
Returns the difference left - right for a sanity check.
"""
function check_symmetry(m::Integer, k::Real)
    left  = besselj(-m, k)
    right = (-1)^m * besselj(m, k)
    return left - right
end

println("Check symmetry condition J_{-m}(k) = (-1)^m J_m(k):")
for m in -2:2
    diff = check_symmetry(m, 5.0)
    println("m = $m, difference = $(diff)")
end
println()

# 2) Check the identity 1 = J_0^2(k) + 2∑_{m=1 to ∞} J_m^2(k)
# We'll do a partial sum up to M = 20 for some k values:
kvals = [0.0, 1.0, 5.0, 10.0]
M = 20
println("Check the sum identity S_M = J_0^2(k) + 2∑_{m=1..M} J_m^2(k), expecting it → 1")
for k in kvals
    s_M = identity_sum_J(k, M)
    println("k = $k, M = $M, partial sum = $s_M")
end
println()

# 3) Check the 2D expansion
# Suppose r=1, r'=2, theta=0.1 rad, theta'=1.0 rad, k=3. We'll do partial sums up to M=10:
r, rprime = 1.0, 2.0
theta, thetaprime = 0.1, 1.0
k = 3.0
M = 10
lhs, rhs_M = bessel_expansion_2D(k, r, rprime, theta, thetaprime, M)
println("Check 2D expansion J_0(k||x-x'||) with M=$M")
println(" - LHS = $lhs")
println(" - RHS (partial sum up to m=$M) = $rhs_M")
println(" - Difference = ", lhs - rhs_M)

Check symmetry condition J_{-m}(k) = (-1)^m J_m(k):
m = -2, difference = 0.0
m = -1, difference = 0.0
m = 0, difference = 0.0
m = 1, difference = 0.0
m = 2, difference = 0.0

Check the sum identity S_M = J_0^2(k) + 2∑_{m=1..M} J_m^2(k), expecting it → 1
k = 0.0, M = 20, partial sum = 1.0
k = 1.0, M = 20, partial sum = 1.0
k = 5.0, M = 20, partial sum = 0.9999999999999999
k = 10.0, M = 20, partial sum = 0.9999999999820721

Check 2D expansion J_0(k||x-x'||) with M=10
 - LHS = -0.25330574971604225
 - RHS (partial sum up to m=10) = -0.25330574313880105
 - Difference = -6.577241196126238e-9


### Equation 50, 51 and 52 

In [11]:
# ---------------------------------------------------------------
# 50) J0(k) via integral definition
#    J0(k) = (2π)^(-1) * ∫[0..2π] e^{ i k cos(θ) } dθ
#
#    Since J0(k) is real, we can integrate cos(k cos θ) instead
#    because  real( e^{ i x } ) = cos(x).
# ---------------------------------------------------------------
function J0_def(k::Real)
    f(θ) = cos(k * cos(θ))   # real part of e^{ i k cos(θ) }
    val, err = quadgk(f, 0, 2π)
    return val / (2π)
end

# ---------------------------------------------------------------
# 50) J1(k) via integral definition
#    J1(k) = (1/k) * ∫[0..k] [ k' J0(k') ] dk'
# ---------------------------------------------------------------
function J1_def(k::Real)
    # For k=0, define J1(0) = 0 by continuity.
    if k == 0
        return 0.0
    end
    
    f(kp) = kp * besselj(0, kp)   # besselj(0, kp) is J0(k')
    val, err = quadgk(f, 0, k)
    return (1/k) * val
end

# ---------------------------------------------------------------
# 51) J_(1/2)(k) closed form
#    J_{1/2}(k) = sqrt(2/(π k)) * sin(k)
# ---------------------------------------------------------------
function J_half_def(k::Real)
    return sqrt(2/(π*k)) * sin(k)
end

# ---------------------------------------------------------------
# 52) Check derivative identity:
#    d/dk [ k J1(k) ] = k J0(k).
#
# We'll verify numerically by finite differences:
#    ( k J1(k) )' ≈ [ k+δ * J1(k+δ) - k * J1(k) ] / δ
# ---------------------------------------------------------------
function check_derivative_identity(k::Real; δ=1e-6)
    # left side: finite difference approximation of d/dk [k J1(k)]
    left = ((k+δ)*besselj(1, k+δ) - k*besselj(1, k)) / δ
    
    # right side: k J0(k)
    right = k * besselj(0, k)
    
    return left - right  # difference should be ~ 0
end

# ---------------------------------------------------------------
# 52) Limits
#    i)   lim_{k -> 0} [ J1(k)/k ] = 0.5
#    ii)  lim_{k -> 0} [ J_m(k)/sqrt(k) ] = 0   (for m>0)
#
# We'll just show approximate evaluations near k=0.
# ---------------------------------------------------------------

function limit_J1_over_k(ε::Real=1e-8)
    return besselj(1, ε)/ε
end

function limit_Jm_over_sqrt_k(m::Integer; ε=1e-8)
    return besselj(m, ε)/sqrt(ε)
end


limit_Jm_over_sqrt_k (generic function with 1 method)

### Sanity check for eq 50, 51, 52

In [12]:

# ---------------------------------------------------------------
# Example Usage & Printouts
# ---------------------------------------------------------------
println("=== 1) Compare J0 via integral with built-in besselj(0, k) ===")
for k in [0.0, 1.0, 5.0]
    j0_int = J0_def(k)
    j0_sf  = besselj(0, k)
    @printf("k = %4.1f | Integral-based J0_def(k) = % .6f | besselj(0,k) = % .6f | Diff = % .6e\n",
            k, j0_int, j0_sf, j0_int - j0_sf)
end
println()

println("=== 2) Compare J1 via integral with built-in besselj(1, k) ===")
for k in [0.0, 1.0, 5.0]
    j1_int = J1_def(k)
    j1_sf  = besselj(1, k)
    @printf("k = %4.1f | Integral-based J1_def(k) = % .6f | besselj(1,k) = % .6f | Diff = % .6e\n",
            k, j1_int, j1_sf, j1_int - j1_sf)
end
println()

println("=== 3) J_(1/2)(k) = √[2/(πk)] sin(k) ===")
for k in [0.5, 1.0, 3.0, 5.0]
    j_half_exact = J_half_def(k)
    # We can also compare with besselj(0.5, k) from SpecialFunctions
    j_half_sf    = besselj(0.5, k)
    @printf("k = %4.1f | J_{1/2} from formula = % .6f | besselj(0.5,k) = % .6f | Diff = % .6e\n",
            k, j_half_exact, j_half_sf, j_half_exact - j_half_sf)
end
println()

println("=== 4) Check derivative identity: d/dk[k J1(k)] ≟ k J0(k) ===")
for k in [0.1, 1.0, 5.0]
    diff = check_derivative_identity(k)
    @printf("k = %4.1f | difference ≈ % .6e\n", k, diff)
end
println()

println("=== 5) Limits as k -> 0 ===")
# (i)  J1(k)/k -> 0.5
val1 = limit_J1_over_k()
@printf("J1(k)/k at k=1e-8 ≈ %g  (expected ~ 0.5)\n", val1)

# (ii) For m>0, J_m(k)/√k -> 0
m_test = 1
val2 = limit_Jm_over_sqrt_k(m_test)
@printf("J%d(k)/√k at k=1e-8 ≈ %g  (expected ~ 0)\n", m_test, val2)

=== 1) Compare J0 via integral with built-in besselj(0, k) ===
k =  0.0 | Integral-based J0_def(k) =  1.000000 | besselj(0,k) =  1.000000 | Diff =  0.000000e+00
k =  1.0 | Integral-based J0_def(k) =  0.765198 | besselj(0,k) =  0.765198 | Diff =  1.110223e-16
k =  5.0 | Integral-based J0_def(k) = -0.177597 | besselj(0,k) = -0.177597 | Diff = -1.387779e-16

=== 2) Compare J1 via integral with built-in besselj(1, k) ===
k =  0.0 | Integral-based J1_def(k) =  0.000000 | besselj(1,k) =  0.000000 | Diff =  0.000000e+00
k =  1.0 | Integral-based J1_def(k) =  0.440051 | besselj(1,k) =  0.440051 | Diff =  0.000000e+00
k =  5.0 | Integral-based J1_def(k) = -0.327579 | besselj(1,k) = -0.327579 | Diff =  0.000000e+00

=== 3) J_(1/2)(k) = √[2/(πk)] sin(k) ===
k =  0.5 | J_{1/2} from formula =  0.540974 | besselj(0.5,k) =  0.540974 | Diff = -5.551115e-16
k =  1.0 | J_{1/2} from formula =  0.671397 | besselj(0.5,k) =  0.671397 | Diff = -6.661338e-16
k =  3.0 | J_{1/2} from formula =  0.065008 | besse

## Equation 54: Kernel:- D(x, x') for k = 1 for now

In [ ]:
"""
    D(x, xprime)

Computes D(x, x') = (K J_1(K ||x - x'||)) / (2π ||x - x'||),
where K = 1, and ||x - x'|| is the Euclidean distance between x and x'.
"""
function D(x::Tuple{Real, Real}, xprime::Tuple{Real, Real})
    # Extract coordinates
    x1, x2 = x
    x1p, x2p = xprime
    
    # Euclidean distance ||x - x'||
    distance = norm([x1 - x1p, x2 - x2p])
    
    # Handle edge case where x == x' (distance = 0)
    if distance == 0
        error("D(x, x') is undefined for x == x' (division by zero).")
    end
    
    # Compute D(x, x')
    K = 1  # Given K = 1
    return (K * besselj(1, K * distance)) / (2π * distance)
end

# Example usage
x = (1.0, 2.0)          # Example point x
xprime = (2.0, 3.0)     # Example point x'
result = D(x, xprime)
println("D(x, x') = $result")
